In [1]:
import ModelManager as mm_mod
import config_defaults as cd
from importlib import reload
import numpy as np
import matplotlib.pyplot as plt
import time
import logging

from tqdm import tqdm_notebook as tqdm


%matplotlib inline
%load_ext autotime

In [2]:
reload(mm_mod)
logger = logging.getLogger('__main__')
logger.setLevel(logging.INFO)
param_overrides = {'NGRAM_MODE': 'naive',
                   'NGRAM_SIZE': 4,
                   'VOC_SIZE': 10000}
mm = mm_mod.ModelManager(hparams=param_overrides, res_name='vocab_explore.p')
mm.load_data()
mm.data_to_pipe()
mm.model_init()  # make sure we force the model to re-init
# mm.training_loop()

2018-10-02 22:11:57,833 __main__     INFO     initialized model with hyperparametrs:
2018-10-02 22:11:57,833 __main__     INFO     LR: 0.01
2018-10-02 22:11:57,834 __main__     INFO     NEPOCH: 10
2018-10-02 22:11:57,834 __main__     INFO     BATCH_SIZE: 32
2018-10-02 22:11:57,834 __main__     INFO     NGRAM_SIZE: 4
2018-10-02 22:11:57,835 __main__     INFO     VOC_SIZE: 10000
2018-10-02 22:11:57,835 __main__     INFO     EMBEDDING_DIM: 100
2018-10-02 22:11:57,836 __main__     INFO     NGRAM_MODE: naive
2018-10-02 22:11:57,836 __main__     INFO     VAL_SIZE: 5000
2018-10-02 22:11:57,837 __main__     INFO     OPTIMIZER: <class 'torch.optim.adam.Adam'>
2018-10-02 22:11:57,837 __main__     INFO     VAL_FREQ: 4
2018-10-02 22:11:57,838 __main__     INFO     REMOVE_STOP_WORDS: True
2018-10-02 22:11:57,838 __main__     INFO     REMOVE_PUNC: True
2018-10-02 22:11:57,839 __main__     INFO     EARLY_STOP: True
2018-10-02 22:11:57,839 __main__     INFO     EARLY_STOP_LOOKBACK: 2
2018-10-02 22:11:

In [3]:
import data_processor as dp
from torch.utils.data import DataLoader

import torch.nn as nn
import torch.nn.functional as tfunc

time: 599 µs


In [4]:
imdb_train = dp.IMDBDataset(mm.data['train'])
imdb_validation = dp.IMDBDataset(mm.data['val'])
imdb_test = dp.IMDBDataset(mm.data['test'])

train_loader = DataLoader(dataset=imdb_train,
                                   batch_size=mm.hparams['BATCH_SIZE'],
                                   collate_fn=dp.imdb_collate_func,
                                   shuffle=True)

val_loader = DataLoader(dataset=imdb_validation,
                                 batch_size=mm.hparams['BATCH_SIZE'],
                                 collate_fn=dp.imdb_collate_func,
                                 shuffle=False)

test_loader = DataLoader(dataset=imdb_test,
                                  batch_size=mm.hparams['BATCH_SIZE'],
                                  collate_fn=dp.imdb_collate_func,
                                  shuffle=False)

time: 8.37 ms


In [5]:
op_constr = mm.hparams['OPTIMIZER']
optimizer = op_constr(mm.model.parameters(), lr=mm.hparams['LR'])
criterion = nn.CrossEntropyLoss()

time: 740 µs


In [6]:
mm.hparams

{'LR': 0.01,
 'NEPOCH': 10,
 'BATCH_SIZE': 32,
 'NGRAM_SIZE': 4,
 'VOC_SIZE': 10000,
 'EMBEDDING_DIM': 100,
 'NGRAM_MODE': 'naive',
 'VAL_SIZE': 5000,
 'OPTIMIZER': torch.optim.adam.Adam,
 'VAL_FREQ': 4,
 'REMOVE_STOP_WORDS': True,
 'REMOVE_PUNC': True,
 'EARLY_STOP': True,
 'EARLY_STOP_LOOKBACK': 2,
 'EARLY_STOP_MIN_IMPROVE': 0.01}

time: 11.6 ms


In [7]:
import BagOfWords as BoW
model = BoW.BagOfWords(len(mm.data['vocab']), mm.hparams['EMBEDDING_DIM']).to(cd.DEVICE)

time: 7.89 ms


In [8]:
for epoch in range(10):
    for i, (data, lengths, labels) in enumerate(train_loader):
        model.train()
        data_batch, length_batch, label_batch = data, lengths, labels
        optimizer.zero_grad()
        outputs = model(data_batch, length_batch)
        loss = criterion(outputs, label_batch)
        loss.backward()
        optimizer.step()

[array([2214,    1,    1, ...,    0,    0,    0]), array([28,  1,  1, ...,  0,  0,  0]), array([   8, 1518,    1, ...,    0,    0,    0]), array([304,   1,   1, ...,   0,   0,   0]), array([2190,    1,    1, ...,    0,    0,    0]), array([1, 1, 1, ..., 0, 0, 0]), array([1, 1, 1, ..., 0, 0, 0]), array([  40, 5525,    1, ...,    0,    0,    0]), array([1, 1, 1, ..., 0, 0, 0]), array([1980,    1,    1, ...,    0,    0,    0]), array([2728,    1,    1, ...,    0,    0,    0]), array([195,   1,   1, ...,   0,   0,   0]), array([122,   1,   1, ...,   0,   0,   0]), array([5093,    1,    1, ...,    0,    0,    0]), array([1391,    1,    1, ...,    0,    0,    0]), array([0., 0., 0., ..., 0., 0., 0.]), array([6581,    1,    1, ...,    0,    0,    0]), array([ 70, 991,   1, ...,   0,   0,   0]), array([1, 1, 1, ..., 0, 0, 0]), array([1, 1, 1, ..., 0, 0, 0]), array([603,   1,   1, ...,   0,   0,   0]), array([1, 1, 1, ..., 0, 0, 0]), array([782,   1,   1, ...,   0,   0,   0]), array([1, 1, 1, .

RuntimeError: Expected tensor for argument #1 'indices' to have scalar type Long; but got CPUDoubleTensor instead (while checking arguments for embedding)

time: 4.07 s


In [9]:
import pickle as pkl

time: 1.72 ms


In [20]:
nparr = pkl.load(open(r'./results/nparr.p', "rb"))

time: 1.21 ms


In [21]:
nparr[24]

array([0., 0., 0., ..., 0., 0., 0.])

time: 2.79 ms


### figuring out which one is the all 0s

In [56]:
import pandas as pd


time: 413 µs


In [60]:
check_data = imdb_train

time: 399 µs


In [62]:
debug_dict = {}
for i, data in enumerate(imdb_train.data_list):
    debug_dict[data.file_name] = len(np.unique(np.array(data.token_idx)))
    
df = pd.DataFrame(debug_dict, index=['count']).T
df.sort_values('count').head()

,count
./data/aclImdb/train/neg/10962_3.txt,0
./data/aclImdb/train/neg/2331_1.txt,2
./data/aclImdb/train/neg/874_1.txt,3
./data/aclImdb/train/pos/4518_9.txt,3
./data/aclImdb/train/neg/9122_1.txt,4


time: 1.19 s


In [82]:
debug_dict = {}
for i, data in enumerate(imdb_train.data_list):
    debug_dict[data.file_name] = len(data.raw_text)
    if data.file_name == './data/aclImdb/train/neg/10962_3.txt':
        err_data = data
    
df = pd.DataFrame(debug_dict, index=['count']).T
df.sort_values('count').head()

,count
./data/aclImdb/train/neg/10962_3.txt,52
./data/aclImdb/train/neg/2331_1.txt,53
./data/aclImdb/train/pos/4518_9.txt,70
./data/aclImdb/train/neg/874_1.txt,80
./data/aclImdb/train/pos/3247_10.txt,81


time: 688 ms


In [83]:
err_data.raw_text

'this movie is terrible but it has some good effects.'

time: 1.24 ms


In [84]:
err_data.tokens

[]

time: 1.05 ms


In [85]:
from ngrams import tokenize

time: 1.72 ms


In [93]:
dataset = tokenize(err_data.raw_text, mode='naive')

time: 473 µs


In [66]:
pickle_path_train_val = r'./data/pickles/trainval_4_naive_True_True.p'
train_and_val_data = pkl.load(open(pickle_path_train_val, "rb"))

time: 3.38 s


In [68]:
import ngrams

time: 1.66 ms


In [70]:
train_ngram_indexer, ngram_counter = ngrams.create_ngram_indexer(train_and_val_data,
                                                                 topk=mm.hparams['VOC_SIZE'],
                                                                 val_size=mm.hparams['VAL_SIZE'])

2018-10-02 22:34:02,466 __main__     INFO     constructing ngram_indexer ...
2018-10-02 22:34:02,473 __main__     INFO     indexer length 20000
2018-10-02 22:34:07,426 __main__     INFO     final vocal size: 10002
time: 4.96 s


In [76]:
len(train_ngram_indexer)

10002

time: 1.28 ms


In [101]:
n=4
from collections import Counter

tokens = tokenize(err_data.raw_text, mode='naive')
print(tokens)

all_ngrams = []
for i in range(0, len(tokens) - n):
    for j in range(1, n + 1):
        print('calling methods for j=%s, i = %s' % (j, i))
        all_ngrams.append(get_n_gram_at_position_i(j, i, tokens))
ngram_counter = Counter(all_ngrams)

ngram_counter, all_ngrams

['movie', 'terrible', 'good', 'effects.']


(Counter(), [])

time: 5.56 ms


In [116]:
tokens2 = tokenize(data.raw_text, mode='naive')

time: 553 µs


In [117]:
def extract_ngram_from_text(text, n, remove_stopwords=True, remove_punc=True, mode='spacy'):
    """
    Function that retrieves all n-grams from the input string
    :param text: raw string
    :param n: integer that tells the model to retrieve all k-gram where k<=n
    :param remove_stopwords: whether or not to remove stopwords from lib
    :param remove_punc: whether or not to remove punctuation from lib
    :param mode: {'spacy', 'naive'}
    :return ngram_counter: a counter that maps n-gram to its frequency
    :return tokens: a list of parsed ngrams
    """
    tokens = tokenize(text, remove_stopwords=remove_stopwords, remove_punc=remove_punc, mode=mode)
    all_ngrams = []
    for i in range(0, len(tokens)):
        all_ngrams.append(get_n_gram_at_position_i(i, n, tokens))
    ngram_counter = Counter(all_ngrams)
    return ngram_counter, all_ngrams

time: 1.3 ms


In [103]:
def get_n_gram_at_position_i(i, n, tokens):
    """ returns list of all n grams at position i"""
    if i + n < len(tokens):
        # can look forward n and be okay
        sub_tokens = tokens[i:i + n]
    else
        sub_tokens = tokens[i:]
    

SyntaxError: unexpected EOF while parsing (<ipython-input-103-b89974563ccc>, line 1)

time: 2.69 ms


In [107]:
tokens[0:6]

['movie', 'terrible', 'good', 'effects.']

time: 3.48 ms


In [108]:
import itertools
def findsubsets(S,m):
    return set(itertools.combinations(S, m))

time: 576 µs


In [124]:
from nltk.util import ngrams as nltk_ngrams

time: 416 µs


In [122]:
list(ngrams(tokens,1))

[('movie',), ('terrible',), ('good',), ('effects.',)]

time: 2.36 ms
